<a href="https://colab.research.google.com/github/yukinaga/ai_voice_writing/blob/main/section_1/01_try_whisper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **DO NOT DISCLOSE YOUR API KEY!!!**

In [ ]:
# OpenAIのAPIキーを取得
API_KEY = ""

# **Google Colab Settings**

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

import os, sys
print('Current Directory: ', os.getcwd())

ROOT_PATH = '/content/drive/My Drive/Colab Notebooks/'
sys.path.append(ROOT_PATH)

DATA_PATH = '/content/drive/My Drive/Colab Notebooks/data/'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
Current Directory:  /content


# **Import Libraries**

- Refrence: OpenAI Documenbt at GitHub
    - https://github.com/openai/openai-python/blob/main/api.md
    - https://github.com/openai/openai-cookbook/blob/main/examples/Whisper_processing_guide.ipynb
    - Api credit is required, but you can try this code for roughly $0.1 or less!

- Dataset : 伊賀市議会「令和５年教育民生常任委員会（12月14日）」
    - https://www.youtube.com/watch?v=NTy5rGt9Yug
    -1. Played the YouTube video and recorded it with voice memo app.
    -2. Saved file named 'rec.wav'.
    -3. The sample file was tried for 2~3 minutes, but can be made longer.

In [ ]:
!pip install git+https://github.com/openai/whisper.git
!pip install --upgrade openai
!pip install pydub
import openai
from openai import OpenAI
from openai.types.audio import Transcription
import whisper

import os
import urllib
import ssl

from IPython.display import Audio
from pathlib import Path

from pydub import AudioSegment
from pydub.silence import detect_nonsilent

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-9gyf45ap
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-9gyf45ap
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.6 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=802825 sha256=26206e2d7f05f8348e5d01c6c5c267d475510344234142260d879e95c54c7aec
  Stored in directory: /tmp/pip-ephem-wheel-cache-pnpe005s/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.9 MB/s eta 0:

# **Simple Transcribe with Whisper (without api)**


In [ ]:
# 音声データの読み込み
audio = whisper.load_audio(DATA_PATH + "rec.wav") #ファイル名はrec.wav
audio = whisper.pad_or_trim(audio)

# 音響特徴量（ログメルスペクトグラム）の計算
mel = whisper.log_mel_spectrogram(audio).cuda()

#言語とモデル規模の設定
lang = "ja"
model = whisper.load_model("medium")

# 音声データの文字起こし
options = whisper.DecodingOptions(language=lang, without_timestamps=True)
result = whisper.decode(model, mel, options)
print(result.text)

# 結果をファイルに書き込む
with open("sample_voice.txt", "w") as f:
    f.write(result.text)

今後どうしていくかというところに今迫られている状況でその中でも教義を示させていただいておりますただ参考を残すために私たちも参考を残していただきたいというのをお願いをしていますそれは先ほども終わったようにいろんな子どもの受けたらおきちんとして心臓補償していただきたいというのを県にお願いをしてきたと思いますところがその中でやはりご校の側からの意見の中ではこれ以上学校規模が小さくなると魅力化すらできないということの意見が出てきてその中で今後どうしていくかということが今議論されています


# **General-purpose model with api**

In [ ]:
# 元の音源を分割
def split_audio(input_file_path, chunk_length_ms=30000):  # 30秒のセグメントに分割
    audio = AudioSegment.from_file(input_file_path, format="wav")
    chunks = []

    for i in range(0, len(audio), chunk_length_ms):
        chunk = audio[i:i+chunk_length_ms]
        chunks.append(chunk)

    return chunks

# 無音部分のトリミング
def trim_silence(audio_path):
    audio = AudioSegment.from_file(audio_path, format="wav")
    non_silent_ranges = detect_nonsilent(audio, min_silence_len=100, silence_thresh=-50) #無音と判断する時間と音量の閾値
    if non_silent_ranges:
        start_trim = non_silent_ranges[0][0] # 開始点：最初の非無音部分のタプル（開始時間, 終了時間）を返す
        end_trim = non_silent_ranges[-1][1] # 終了点：最後の非無音部分のタプル（開始時間, 終了時間）を返す
        trimmed_audio = audio[start_trim:end_trim]
    else:
        trimmed_audio = audio

    # トリミングしたオーディオを一時ファイルに保存
    trimmed_file_path = "./trimmed.wav"
    trimmed_audio.export(trimmed_file_path, format="wav")
    return trimmed_file_path


# ビットレートの調整（ファイルサイズの削減）
import subprocess
def reduce_bitrate(input_file_path, output_file_path, target_bitrate="64k", target_sample_rate="22050"):

    #file_size_input = os.path.getsize(input_file_path)
    #file_size_output = os.path.getsize(output_file_path)
    #print("File Size in Bytes:", file_size_input, file_size_output)

    # ffmpegコマンドを使用してビットレートを変更
    command = [
        "ffmpeg",
        "-y",  # 既存のファイルを上書きすることを許可
        "-i", input_file_path,  # 入力ファイル
        "-b:a", target_bitrate,  # オーディオビットレートの設定
        "-ar", target_sample_rate,  # サンプリングレートの設定
        output_file_path  # 出力ファイル
    ]
    try:
        # 標準エラー出力をキャプチャしてエラー発生時に表示
        subprocess.run(command, check=True, stderr=subprocess.PIPE, stdout=subprocess.PIPE)
    except subprocess.CalledProcessError as e:
        # エラーメッセージを出力
        print(f"Error occurred: {e.stderr.decode()}")
        raise


def transcribe_audio(input_file_path):
    # 無音部分をトリミングし、その結果のパスを取得
    trimmed_file_path = trim_silence(input_file_path) # 原音→トリミング

    # ビットレートを下げたファイルを保存するパス
    reduced_file_path = "./reduced.wav"

    # トリミングされ、ビットレートが下げられたファイルを作成
    reduce_bitrate(trimmed_file_path, reduced_file_path) # トリミング→圧縮

    # 文字起こしを行う
    # ファイルをバイナリモードで開き、その内容をAPIに渡す
    with open(reduced_file_path, 'rb') as audio_file:  #圧縮 → 音源
        transcription_params = {
            "file": audio_file,
            "model": "whisper-1",
            "language": "ja",
        }
        # API呼び出しでの正しいメソッド名とパラメータを使用
        response = client.audio.transcriptions.create(**transcription_params) # 音源 → 文字起こし
        #print(type(response))
        #print(dir(response))
        transcribed_text = response.text
        # レスポンスから文字起こし結果を抽出
        #transcribed_text = response["choices"][0]["text"]
        return transcribed_text


def transcribe_chunks(chunks):
    transcriptions = []

    for i, chunk in enumerate(chunks):

        chunk_file_path = f"./temp_chunk_{i}.wav"
        chunk.export(chunk_file_path, format="wav") #一時ファイルに保存

        transcription = transcribe_audio(chunk_file_path)
        transcription_cleaned = transcription.replace("\n", " ") #改行を削除
        transcriptions.append(transcription_cleaned)
        os.remove(chunk_file_path) #使用後の一時ファイルを削除

    return transcriptions

# APIキーによる認証
client = OpenAI(api_key=API_KEY)

original_file_path = DATA_PATH + "rec.wav"
chunks = split_audio(original_file_path)
transcriptions = transcribe_chunks(chunks)

print(transcriptions)


['いうことで 今後どうしていくか というところに 今 迫られている 状況で その中でも 協議を示させて いただいております ただ 参考 を残すという 私たちもこれまで 参考 私もですけど 参考を残して いただきたいというのは お願いを してきました それは 先ほども 終わったように いろんなことを 物を受けたらおきちとして 進路 補助をしていただきたいという のを 県にお願いをしてきたと思います ところが その中で やはり 結局 国防側からの意見の中では これ以上 学校規模が小さくなると 魅力化 すらできないということの意見 が出てきて その中で 今後どうして いくかということが 今 議論されて ', '今後も議論をされていくという中で今あるというような状況でございまして 今後子供が減る何年かの段階の中でどうしていくのかを今論議をして その中で決定しているただですね論議はしますけれども最終決定は金教委ですので 論議をしていただいて金教委の中で最終判断をしているというところへ 今来ているというのが現状でございます 実情はよく分かりましたので中学校の進路担当の方も大変悩んでるやろうし 受け入れていく高校の校長先生か教員の方も大変悩ましいことやろうと そういう認識を', '引き続いて、保護者、地域の考え方も伝えていただきながら 進路の確保ということで、教育委員会も取り組んでいただきたいと思います 何かございませんか 今、北山委員からも、イムカセーカ協議会ということの お話の現状のことと、教育庁からも、今議論中であるということも 聞かせていただいたんですけれども', '県立高校のペースのあり方とか そういったものに制限されているわけなんですけども PTAの連合会とか教職組合への対応というのは どういうふうになっているのでしょうか 【佐藤】市教育長 【佐藤】PTA連合会 それから教職組合等ということでございますけれども その以外 地域の高等学校の活性化推進協議会の メンバーの中にですね 小中学校のPTA連合会の方の代表の方 それから高校のPTAの代表の方が出てくれてますし それから教育委員会としては 両教育長が', 'ながらに両教育長が出ておりますし 学校の代表もですね 校長の代表も出ています さらには教職員の代表の方も出ていますので それで高等学校の校長も出ておりますけど

# **Summerize with ChatGPT**

In [ ]:
def summarize_text(text, system_message, prompt):

    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": system_message}, #role:役割とcontent:条件の指定
            {"role": "user", "content": prompt}
        ],
        temperature=0.1,  # クリエイティブさの度合いを調整
        max_tokens=1000  # 生成するトークンの最大数を指定（プロンプトでコントロール / 文字数ではない）
    )

    # APIレスポンスから要約されたテキストを得る
    # そのうち最初の選択肢(choice[0])のレスポンス(message)と要約文章(content)を取得する
    summarized_text = completion.choices[0].message.content
    return summarized_text


# GPTに要約を依頼する際の要件とプロンプト文を設定
system_message = '''
以下の要件に基づき、文章を整理し議事録としてまとめてください。

【要件】
* 現状を把握し、話の全体像がわかるように250文字以内で要約してください。
*　個人の主張を50文字以内で5つまで記述してください。その時名前を含めてください。
* 今後取るべき具体的なアクションを100文字以内で記述してください。
* 議題を1つ100文字以内で記述してください。
* 採決が行われた場合、その内容と結果（賛成票と反対票の数）を記述してください。
* 議事録以外の文章は記述しないでください。
'''

prompt = f"以下のテキストを要約してください:\n\n{transcriptions}"

# transcribe_audioで得られたテキストを要約
summarized_text = summarize_text(transcriptions, system_message, prompt)
print(summarized_text)

要約：
現在、学校の規模や進路に関する協議が行われており、子供の減少による影響や地域の活性化について検討されている。教育委員会や地域の関係者が参加し、最終決定は教育委員会が行う。各学校の定員削減や地域貢献についても話し合われている。

個人の主張：
1. 【佐藤】: "学校の規模を小さくすることも検討すべき。"
2. 【上杉】: "他校の存続がどのように影響するか考慮すべき。"
3. 【小野】: "地域活性化への貢献を重視し、学校の存在価値を考えるべき。"
4. 【藍澤】: "学校の存在が地域の活力につながると考える。"
5. 【加藤】: "地域貢献活動を通じて地域の活性化に貢献している。"

アクション：
学校の定員削減や地域貢献活動を進め、教育委員会と地域の連携を強化して、地域の活性化と学校の存続を促進する。

議題：
学校の規模や進路に関する協議と地域活性化についての検討。

採決：
採決は行われていない。
